<a href="https://colab.research.google.com/github/fahernkhan/Natural-Language-Processing-NLP-/blob/main/NLP%20for%20News%20Multiclass%20Categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

dataset : https://www.kaggle.com/datasets/hgultekin/bbcnewsarchive

In [ ]:
import pandas as pd

df = pd.read_csv('bbc-news-data.csv', sep='\t')
df.head()

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [ ]:
df.shape

(2225, 4)

In [ ]:
df = df.drop(columns=['filename'])

In [ ]:
df.head()

,category,title,content
0,business,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [ ]:
category = pd.get_dummies(df.category)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='category')
df_baru

,title,content,business,entertainment,politics,sport,tech
0,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...,1,0,0,0,0
1,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...,1,0,0,0,0
2,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...,1,0,0,0,0
3,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...,1,0,0,0,0
4,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...,1,0,0,0,0
...,...,...,...,...,...,...,...
2220,BT program to beat dialler scams,BT is introducing two initiatives to help bea...,0,0,0,0,1
2221,Spam e-mails tempt net shoppers,Computer users across the world continue to i...,0,0,0,0,1
2222,Be careful how you code,A new European directive could put software w...,0,0,0,0,1
2223,US cyber security chief resigns,The man making sure US computer networks are ...,0,0,0,0,1


In [ ]:
news = df_baru['title'].values + '' + df_baru['content'].values
label = df_baru[['business', 'entertainment', 'politics', 'sport', 'tech']].values

In [ ]:
from sklearn.model_selection import train_test_split
news_latih, news_test, label_latih, label_test = train_test_split(news, label, test_size=0.2)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='x')
tokenizer.fit_on_texts(news_latih) 
tokenizer.fit_on_texts(news_test)
 
sekuens_latih = tokenizer.texts_to_sequences(news_latih)
sekuens_test = tokenizer.texts_to_sequences(news_test)
 
padded_latih = pad_sequences(sekuens_latih) 
padded_test = pad_sequences(sekuens_test)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.9 and logs.get('val_accuracy')>0.9):
      self.model.stop_training = True
      print("\nakurasi sudah mencapai 90%!")
callbacks = myCallback()

In [ ]:
num_epochs = 30
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
                    validation_data=(padded_test, label_test), verbose=2,
                    callbacks=[callbacks], validation_steps=30)

Epoch 1/30
56/56 - 25s - loss: 1.6050 - accuracy: 0.2292 - val_loss: 1.5896 - val_accuracy: 0.2315 - 25s/epoch - 444ms/step
Epoch 2/30
56/56 - 14s - loss: 1.3591 - accuracy: 0.3747 - val_loss: 1.1385 - val_accuracy: 0.4090 - 14s/epoch - 258ms/step
Epoch 3/30
56/56 - 14s - loss: 0.9776 - accuracy: 0.5067 - val_loss: 1.1525 - val_accuracy: 0.4045 - 14s/epoch - 255ms/step
Epoch 4/30
56/56 - 14s - loss: 0.6887 - accuracy: 0.7006 - val_loss: 0.7411 - val_accuracy: 0.6472 - 14s/epoch - 253ms/step
Epoch 5/30
56/56 - 13s - loss: 0.4545 - accuracy: 0.8315 - val_loss: 0.8677 - val_accuracy: 0.6539 - 13s/epoch - 233ms/step
Epoch 6/30
56/56 - 13s - loss: 0.2258 - accuracy: 0.9298 - val_loss: 0.7049 - val_accuracy: 0.7506 - 13s/epoch - 235ms/step
Epoch 7/30
56/56 - 11s - loss: 0.0937 - accuracy: 0.9702 - val_loss: 0.7734 - val_accuracy: 0.7506 - 11s/epoch - 198ms/step
Epoch 8/30
56/56 - 12s - loss: 0.0435 - accuracy: 0.9910 - val_loss: 0.7780 - val_accuracy: 0.7820 - 12s/epoch - 211ms/step
Epoch 9/